# ibdb Tutorial

In [1]:
from ib_insync import util, IB, Forex
from sqlalchemy import create_engine
import ibdb

connect using [ib-insync](https://github.com/erdewit/ib_insync)

In [2]:
util.startLoop()  # Jupyter Notebook only

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=3)

<IB connected to 127.0.0.1:4002 clientId=3>

create engine your database drivers

In [3]:
engine = create_engine('sqlite:///:memory:')

### Bar Data

In [4]:
contract = Forex('EURUSD')
bar_eurusd = ib.reqHistoricalData(
    contract,
    endDateTime='',
    durationStr='2 D',
    barSizeSetting='1 hour',
    whatToShow='MIDPOINT',
    useRTH=True)

In [5]:
bar_eurusd[-5:]

[BarData(date=datetime.datetime(2019, 2, 12, 18, 0), open=1.126595, high=1.127635, low=1.126255, close=1.127435, volume=-1, barCount=-1, average=-1.0),
 BarData(date=datetime.datetime(2019, 2, 12, 19, 0), open=1.127435, high=1.128645, low=1.1273, close=1.128615, volume=-1, barCount=-1, average=-1.0),
 BarData(date=datetime.datetime(2019, 2, 12, 20, 0), open=1.128615, high=1.12863, low=1.12741, close=1.128245, volume=-1, barCount=-1, average=-1.0),
 BarData(date=datetime.datetime(2019, 2, 12, 21, 0), open=1.128245, high=1.129565, low=1.128025, close=1.128935, volume=-1, barCount=-1, average=-1.0),
 BarData(date=datetime.datetime(2019, 2, 12, 22, 0), open=1.128935, high=1.12977, low=1.128875, close=1.129635, volume=-1, barCount=-1, average=-1.0)]

specify `sqlalchemy.engine.base.Engine` and `tablename`

In [6]:
bar_table_eurusd = ibdb.tables.Bars(engine, "EURUSD")

create table

In [7]:
bar_table_eurusd.create_table()

insert data

In [8]:
bar_table_eurusd.insert(bar_eurusd)

select by SQL

In [9]:
import pandas as pd

pd.read_sql("select * from EURUSD limit 5", engine)

,date,open,high,low,close,volume,barCount,average
0,2019-02-11 07:15:00.000000,1.131375,1.132475,1.129925,1.131895,-1,-1,-1.0
1,2019-02-11 08:00:00.000000,1.131895,1.132500,1.131875,1.132390,-1,-1,-1.0
2,2019-02-11 09:00:00.000000,1.132390,1.132555,1.132280,1.132345,-1,-1,-1.0
3,2019-02-11 10:00:00.000000,1.132345,1.132775,1.132345,1.132735,-1,-1,-1.0
4,2019-02-11 11:00:00.000000,1.132735,1.132775,1.132395,1.132445,-1,-1,-1.0


### Tick Data

In [10]:
import datetime

contract = Forex('USDJPY')
start = ''
end = datetime.datetime.now()
tick_usdjpy = ib.reqHistoricalTicks(contract, start, end, 10, whatToShow="BID_ASK",useRth=False)

In [11]:
tick_usdjpy[-5:]

[HistoricalTickBidAsk(time=datetime.datetime(2019, 2, 12, 13, 20, 42, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=110.382, priceAsk=110.383, sizeBid=10000000, sizeAsk=2000000),
 HistoricalTickBidAsk(time=datetime.datetime(2019, 2, 12, 13, 20, 42, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=110.382, priceAsk=110.383, sizeBid=9000000, sizeAsk=2000000),
 HistoricalTickBidAsk(time=datetime.datetime(2019, 2, 12, 13, 20, 42, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=110.382, priceAsk=110.383, sizeBid=9000000, sizeAsk=1000000),
 HistoricalTickBidAsk(time=datetime.datetime(2019, 2, 12, 13, 20, 42, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=110.382, priceAsk=110.383, sizeBid=9000000, sizeAsk=2000000),
 HistoricalTickBidAsk(time=datetime.datetime(2019, 2, 12, 13, 20, 42, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=110.382, priceAs

In [12]:
tick_table_usdjpy = ibdb.tables.Ticks(engine, "USDJPY")

In [13]:
tick_table_usdjpy.create_table()

In [14]:
tick_table_usdjpy.insert(tick_usdjpy)

In [15]:
pd.read_sql("select * from USDJPY", engine)

,id,time,priceBid,priceAsk,sizeBid,sizeAsk
0,828247bbc937750bb19c73bab44d90fa,2019-02-12 13:20:41.000000,110.383,110.384,3000000.0,1000000.0
1,191ccccd13026310dd0a1cf75b524661,2019-02-12 13:20:42.000000,110.383,110.384,2000000.0,1000000.0
2,7af19b3a18cf4eaa66b03ddf88028543,2019-02-12 13:20:42.000000,110.383,110.384,4000000.0,1000000.0
3,510b74ed3afc3c28035349b099b615db,2019-02-12 13:20:42.000000,110.383,110.384,3000000.0,2000000.0
4,0d06a00df12652b9b1426b3e153335e6,2019-02-12 13:20:42.000000,110.383,110.384,3000000.0,3000000.0
5,5a4b200bb71c3a36b955e32668d48635,2019-02-12 13:20:42.000000,110.382,110.384,11000000.0,3000000.0
6,9bbfa37ff29cc90f506e8f86c1430029,2019-02-12 13:20:42.000000,110.382,110.384,11000000.0,4000000.0
7,97b52376aa56ef97d0e523059d44868e,2019-02-12 13:20:42.000000,110.382,110.384,11000000.0,5000000.0
8,7f2149e066d69f07116dc04c2956f559,2019-02-12 13:20:42.000000,110.382,110.383,10000000.0,1000000.0
9,18c827453eb8113520b934fd826e9a74,2019-02-12 13:20:42.000000,110.382,110.383,10000000.0,2000000.0


### Fills (Trade History)

In [16]:
fills = ib.fills()

In [17]:
fills[-3:]

[Fill(contract=Option(conId=288525886, symbol='N225', lastTradeDateOrContractMonth='20190307', strike=18750.0, right='P', multiplier='1000', exchange='OSE.JPN', currency='JPY', localSymbol='134038718', tradingClass='NK225'), execution=Execution(execId='0000fa22.5c61fb7b.01.01', time=datetime.datetime(2019, 2, 12, 9, 29, 49, tzinfo=datetime.timezone.utc), acctNumber='U544417', exchange='OSE.JPN', side='SLD', shares=1.0, price=25.0, permId=358478949, cumQty=5.0, avgPrice=25.0, lastLiquidity=1), commissionReport=CommissionReport(), time=datetime.datetime(2019, 2, 12, 9, 29, 49, tzinfo=datetime.timezone.utc)),
 Fill(contract=Option(conId=288525831, symbol='N225', lastTradeDateOrContractMonth='20190307', strike=17000.0, right='P', multiplier='1000', exchange='OSE.JPN', currency='JPY', localSymbol='134037018', tradingClass='NK225'), execution=Execution(execId='0000fa22.5c61fb7d.01.01', time=datetime.datetime(2019, 2, 12, 9, 33, 37, tzinfo=datetime.timezone.utc), acctNumber='U544417', exchang

In [18]:
fills_table = ibdb.tables.Fills(engine, "trade_log")

In [19]:
fills_table.create_table()

In [20]:
fills_table.insert(fills)

In [21]:
pd.read_sql("select * from trade_log limit 5", engine)

,execId,time,acctNumber,exchange,side,shares,price,permId,clientId,orderId,...,includeExpired,secIdType,secId,comboLegsDescrip,comboLegs,deltaNeutralContract,commission,realizedPNL,yield_,yieldRedemptionDate
0,0000fa22.5c61f864.01.01,2019-02-12 00:28:22.000000,U544417,OSE.JPN,SLD,10.0,5.0,358478869,0,0,...,0,,,,None,None,0.0,0.0,0.0,0.0
1,0000fa22.5c61f865.01.01,2019-02-12 00:28:58.000000,U544417,OSE.JPN,BOT,1.0,95.0,358478871,0,0,...,0,,,,None,None,0.0,0.0,0.0,0.0
2,0000fa22.5c61f866.01.01,2019-02-12 00:29:07.000000,U544417,OSE.JPN,SLD,4.0,9.0,358478872,0,0,...,0,,,,None,None,0.0,0.0,0.0,0.0
3,0000fa22.5c61f875.01.01,2019-02-12 00:32:40.000000,U544417,OSE.JPN,BOT,1.0,92.0,358478874,0,0,...,0,,,,None,None,0.0,0.0,0.0,0.0
4,0000fa22.5c61f87d.01.01,2019-02-12 00:33:01.000000,U544417,OSE.JPN,BOT,4.0,25.0,358478873,0,0,...,0,,,,None,None,0.0,0.0,0.0,0.0


In [22]:
ib.disconnect()
engine.dispose()